In [14]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

pd.set_option('display.max_columns', None)

In [8]:
DATA_PATH = r'football_players_dataset.csv'

In [9]:
df = pd.read_csv(DATA_PATH)

In [10]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9680 entries, 0 to 9679
Data columns (total 70 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   player_id                       9680 non-null   object 
 1   player_name                     9680 non-null   object 
 2   age                             9680 non-null   float64
 3   nationality                     9666 non-null   object 
 4   height                          9680 non-null   float64
 5   foot                            6802 non-null   object 
 6   position                        9680 non-null   object 
 7   current_club                    9680 non-null   object 
 8   league                          9680 non-null   object 
 9   appearances                     9680 non-null   int64  
 10  minutes_played                  9680 non-null   int64  
 11  minutes_per_game                9437 non-null   float64
 12  goals                           96

In [12]:
column_descriptions = {
    # ===== THÔNG TIN CƠ BẢN =====
    'player_id': 'ID duy nhất của cầu thủ',
    'player_name': 'Tên cầu thủ',
    'age': 'Tuổi của cầu thủ (năm)',
    'nationality': 'Quốc tịch của cầu thủ',
    'height': 'Chiều cao của cầu thủ (cm)',
    'foot': 'Chân thuận (Left/Right/Both)',
    'position': 'Vị trí thi đấu chính',
    'current_club': 'Câu lạc bộ hiện tại',
    'league': 'Giải đấu hiện tại',

    # ===== THỜI GIAN THI ĐẤU =====
    'minutes_played': 'Tổng số phút thi đấu',
    'minutes_per_game': 'Số phút thi đấu trung bình mỗi trận',

    # ===== CHỈ SỐ TẤN CÔNG =====
    'goals': 'Tổng số bàn thắng',
    'assists': 'Tổng số kiến tạo',
    'goals_per_90': 'Số bàn thắng trung bình mỗi 90 phút',
    'assists_per_90': 'Số kiến tạo trung bình mỗi 90 phút',
    'npg_per90': 'Số bàn thắng không tính penalty mỗi 90 phút',
    'npxg_per90': 'Chỉ số xG không tính penalty mỗi 90 phút',
    'xag_per90': 'Chỉ số kiến tạo kỳ vọng (xAG) mỗi 90 phút',
    'npxg_xag_per90': 'Tổng npxG + xAG mỗi 90 phút',
    'xg_per90': 'Chỉ số bàn thắng kỳ vọng (xG) mỗi 90 phút',
    'shots_per90': 'Số cú sút trung bình mỗi 90 phút',
    'shots_on_target_per90': 'Số cú sút trúng đích mỗi 90 phút',
    'shots_on_target_pct': 'Tỷ lệ sút trúng đích (%)',
    'avg_shot_distance': 'Khoảng cách sút trung bình (m)',
    'sca_per90': 'Số hành động tạo cơ hội dứt điểm mỗi 90 phút',
    'gca_per90': 'Số hành động tạo bàn thắng mỗi 90 phút',

    # ===== CHỈ SỐ CHUYỀN BÓNG – KIẾN TẠO =====
    'key_passes_per90': 'Số đường chuyền tạo cơ hội mỗi 90 phút',
    'passes_completed_per90': 'Số đường chuyền chính xác mỗi 90 phút',
    'pass_completion_pct': 'Tỷ lệ chuyền bóng chính xác (%)',
    'passes_into_final_third_per90': 'Số đường chuyền vào 1/3 cuối sân mỗi 90 phút',
    'passes_into_penalty_area_per90': 'Số đường chuyền vào vòng cấm mỗi 90 phút',
    'progressive_passes_per90': 'Số đường chuyền tịnh tiến mỗi 90 phút',
    'progressive_passes_rec_per90': 'Số lần nhận bóng tịnh tiến mỗi 90 phút',
    'progressive_carries_per90': 'Số lần dẫn bóng tịnh tiến mỗi 90 phút',

    # ===== CHỈ SỐ TRIỂN KHAI BÓNG – RÊ DẮT =====
    'take_ons_per90': 'Số lần rê bóng qua người mỗi 90 phút',
    'take_on_success_pct': 'Tỷ lệ rê bóng thành công (%)',
    'carries_per90': 'Số lần dẫn bóng mỗi 90 phút',
    'carries_into_final_third_per90': 'Số lần dẫn bóng vào 1/3 cuối sân mỗi 90 phút',
    'touches_per90': 'Số lần chạm bóng mỗi 90 phút',
    'touches_att_third_per90': 'Số lần chạm bóng ở 1/3 tấn công mỗi 90 phút',
    'touches_att_pen_per90': 'Số lần chạm bóng trong vòng cấm đối phương mỗi 90 phút',
    'passes_received_per90': 'Số lần nhận đường chuyền mỗi 90 phút',

    # ===== CHỈ SỐ PHÒNG NGỰ =====
    'tackles_per90': 'Số pha tắc bóng mỗi 90 phút',
    'interceptions_per90': 'Số pha cắt bóng mỗi 90 phút',
    'blocks_per90': 'Số pha chắn bóng mỗi 90 phút',
    'ball_recoveries_per90': 'Số lần thu hồi bóng mỗi 90 phút',

    # ===== CHỈ SỐ KHÔNG CHIẾN – THỂ CHẤT =====
    'aerials_won_per90': 'Số pha không chiến thắng mỗi 90 phút',
    'aerial_win_pct': 'Tỷ lệ không chiến thắng (%)',

    # ===== CHỈ SỐ KỶ LUẬT =====
    'yellow_cards_per90': 'Số thẻ vàng mỗi 90 phút',
    'red_cards_per90': 'Số thẻ đỏ mỗi 90 phút',
    'fouls_committed_per90': 'Số lần phạm lỗi mỗi 90 phút',

    # ===== CHỈ SỐ THỦ MÔN – CẢN PHÁ =====
    'goals_against_per90': 'Số bàn thua trung bình mỗi 90 phút',
    'shots_on_target_against_per90': 'Số cú sút trúng đích phải nhận mỗi 90 phút',
    'saves_per90': 'Số pha cứu thua mỗi 90 phút',
    'save_percentage': 'Tỷ lệ cứu thua (%)',
    'clean_sheet_pct': 'Tỷ lệ giữ sạch lưới (%)',
    'psxg_per_shot': 'Chỉ số PSxG trung bình mỗi cú sút phải nhận',
    'psxg_ga_per90': 'PSxG trừ bàn thua mỗi 90 phút',
    'penalty_save_pct': 'Tỷ lệ cản phá penalty (%)',

    # ===== CHỈ SỐ THỦ MÔN – PHÁT BÓNG & CHUYỀN BÓNG =====
    'passes_attempted_per90': 'Số đường chuyền thực hiện mỗi 90 phút',
    'launch_pct': 'Tỷ lệ phát bóng dài (%)',
    'avg_pass_length': 'Độ dài đường chuyền trung bình (m)',

    # ===== CHỈ SỐ THỦ MÔN – QUÉT BÓNG =====
    'def_actions_outside_pen_per90': 'Số pha phòng ngự ngoài vòng cấm mỗi 90 phút',
    'avg_distance_def_actions': 'Khoảng cách trung bình của các pha phòng ngự (m)',

    # ===== CHỈ SỐ THỦ MÔN – CHỐNG BÓNG BỔNG =====
    'crosses_stopped_pct': 'Tỷ lệ cản phá bóng bổng từ tạt cánh (%)',

    # ===== CHỈ SỐ KẾT QUẢ TRẬN ĐẤU =====
    'wins_per90': 'Số trận thắng trung bình mỗi 90 phút',
    'draws_per90': 'Số trận hòa trung bình mỗi 90 phút',
    'losses_per90': 'Số trận thua trung bình mỗi 90 phút',

    # ===== GIÁ TRỊ THỊ TRƯỜNG (TARGET) =====
    'market_value': 'Giá trị thị trường ước tính của cầu thủ (EUR)'
}


In [11]:
df.position.unique()

array(['FW', 'FW-MF', 'MF', 'DF', 'DF-MF', 'DF-FW', 'GK', 'DF-FW-MF'],
      dtype=object)

In [20]:
df[df.minutes_per_game.isna()]

,player_id,player_name,age,nationality,height,foot,position,current_club,league,appearances,minutes_played,minutes_per_game,goals,assists,goals_per_90,assists_per_90,npg_per90,npxg_per90,xag_per90,npxg_xag_per90,xg_per90,shots_per90,shots_on_target_per90,shots_on_target_pct,avg_shot_distance,sca_per90,gca_per90,key_passes_per90,passes_completed_per90,pass_completion_pct,passes_into_final_third_per90,passes_into_penalty_area_per90,progressive_passes_per90,progressive_passes_rec_per90,progressive_carries_per90,take_ons_per90,take_on_success_pct,carries_per90,carries_into_final_third_per90,touches_per90,touches_att_third_per90,touches_att_pen_per90,passes_received_per90,tackles_per90,interceptions_per90,blocks_per90,ball_recoveries_per90,aerials_won_per90,aerial_win_pct,yellow_cards_per90,red_cards_per90,fouls_committed_per90,goals_against_per90,shots_on_target_against_per90,saves_per90,save_percentage,clean_sheet_pct,psxg_per_shot,psxg_ga_per90,penalty_save_pct,passes_attempted_per90,launch_pct,avg_pass_length,def_actions_outside_pen_per90,avg_distance_def_actions,crosses_stopped_pct,wins_per90,draws_per90,losses_per90,market_value


In [19]:
df['minutes_per_game'] = df['minutes_per_game'].fillna(0)

In [ ]:
df.drop